
# <span style="color:Blue"> Apache Spark - Handle Key Value in Complex JSON </span>
######
## Discuss on Couple of Approach
#

    - Method 1: Simple Straight Forward Method
    - Method 2: Optimized method to get results faster

## Understand the optimized way to get results

In [0]:
{
  "name":"azarudeen",
  "grade":"high school",
  "marks":[
    { "key":"Science", "value":88 },
    { "key":"English", "value":74 },
    { "key":"Maths", "value":100  }
  ]
}
{
  "name":"karthick",  
  "grade":"BE graduate", 
  "marks":[
    {"key":"Analytics","value":73},
    {"key":"Dynamics","value":80}
    ]}
{
  "name":"Sivakumar", 
  "grade":"high school", 
  "marks":[
    {"key":"computer","value":69}, 
    {"key":"English","value":64},
    {"key":"Maths","value":90}
    ]}

Out[1]: {'name': 'Sivakumar',
 'grade': 'high school',
 'marks': [{'key': 'computer', 'value': 69},
  {'key': 'English', 'value': 64},
  {'key': 'Maths', 'value': 90}]}

In [0]:
%fs ls dbfs:/FileStore/shared_uploads/azar.s91@gmail.com/keyTest.json

path,name,size,modificationTime
dbfs:/FileStore/shared_uploads/azar.s91@gmail.com/keyTest.json,keyTest.json,402,1707646887000


In [0]:
%fs head dbfs:/FileStore/shared_uploads/azar.s91@gmail.com/keyTest.json

{"name":"azarudeen", "grade":"high school", "marks":[{"key":"Science","value":88}, {"key":"English","value":74},{"key":"Maths","value":100}]}
{"name":"karthick", "grade":"BE graduate", "marks":[{"key":"Analytics","value":73},{"key":"Dynamics","value":80}]}
{"name":"Sivakumar", "grade":"high school", "marks":[{"key":"computer","value":69}, {"key":"English","value":64},{"key":"Maths","value":90}]}

In [0]:
in_path="dbfs:/FileStore/shared_uploads/azar.s91@gmail.com/keyTest.json"
df = spark.read.option("multiLine",False).json(in_path)
display(df)

grade,marks,name
high school,"List(List(Science, 88), List(English, 74), List(Maths, 100))",azarudeen
BE graduate,"List(List(Analytics, 73), List(Dynamics, 80))",karthick
high school,"List(List(computer, 69), List(English, 64), List(Maths, 90))",Sivakumar



## Method - 1: Use Explode and Pivot 

### Steps
  1) ```Explode the given list;```
  2) ```Expand the out to multiple column;```
  3) ```Pivot on Key and agg on value;```

In [0]:
from pyspark.sql.functions import explode_outer,first

(
  # step 1 - Explode the Marks column to convert single record to multiple record 
  df.withColumn("json_key_val",explode_outer("marks"))
    
    # Step 2 - Expand the struct column created above to split to  multiple cols
    .select("*","json_key_val.*")

    # Step 3 - Pivot Key col and agg on value field
    .groupBy("name","grade")
    .pivot("key")
    .agg(first("value"))

    # Step 4 - Drop unnecessary fields
    .drop("json_key_val","marks")

).display()

name,grade,Analytics,Dynamics,English,Maths,Science,computer
Sivakumar,high school,null,null,64,90,null,69
azarudeen,high school,null,null,74,100,88,null
karthick,BE graduate,73,80,null,null,null,null



## Method - 2: Use Map_From_Array and explode

### Steps
  1) ```Get list of key columns and values columns seperated;```
  2) ```Use Map From Array and pass key and value col to get MapType;```
  3) ```Create list of columns in the key field;```
  4) ```Select the fields required;```

In [0]:
from pyspark.sql.functions import col,map_from_arrays,explode,coalesce,expr,lower

mappedDF=(
     # Step 1 - Get Key/Values from the Array of Key value pairs
  df.select("*"
            ,col("marks").getField("key").alias("event_keys")
            ,col("marks").getField("value").alias("event_values")
           )
  
     # Step 2 - Use Map from Array function to form JSON as we expect 
    .select("*"
            ,map_from_arrays(expr("transform(event_keys, x -> lower(x))"), col("event_values")).alias("event_col")
           )
    .drop("marks")

)

# Step 3 - Get all the key columns to select all cols
event_keysDF = mappedDF.select(explode("event_keys")).distinct()

event_key_col_list=[]
for x in event_keysDF.collect():
  event_key_col_list.append(col('event_col.'+x[0].lower()).alias(x[0]))

#Step 4 - Select all the required columns from final Mapped DF
mappedDF.select("name","grade",*event_key_col_list).display()

name,grade,Science,Dynamics,English,Analytics,computer,Maths
azarudeen,high school,88,null,74,null,null,100
karthick,BE graduate,null,80,null,73,null,null
Sivakumar,high school,null,null,64,null,69,90


In [0]:
mappedDF.select(explode("event_keys")).display()

In [0]:
mappedDF.display()

grade,name,event_keys,event_values,event_col
high school,azarudeen,"List(Science, English, Maths)","List(88, 74, 100)","Map(science -> 88, english -> 74, maths -> 100)"
BE graduate,karthick,"List(Analytics, Dynamics)","List(73, 80)","Map(analytics -> 73, dynamics -> 80)"
high school,Sivakumar,"List(computer, English, Maths)","List(69, 64, 90)","Map(computer -> 69, english -> 64, maths -> 90)"
